In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import ElasticNetCV
from sklearn.metrics import mean_squared_error
import numpy as np
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from mlxtend.regressor import StackingCVRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.metrics import r2_score
import multiprocessing
import optuna
from sklearn.model_selection import cross_val_score
import cma

Only `cma.purecma` has been imported. Install `numpy` ("pip install numpy") if you want to import the entire `cma` package.


c:\Users\user\anaconda3\envs\DACON-AI\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
m = pd.read_csv("../../data/processed/processed_3.csv")

In [3]:
m

,감독,제작사,배급사,전국 스크린수,전국 관객수,국적_기타,국적_독일,국적_러시아,국적_미국,국적_스페인,...,장르_어드벤처,장르_전쟁,장르_코미디,장르_판타지,등급_12세이상관람가,등급_15세이상관람가,등급_전체관람가,등급_청소년관람불가,영화구분_독립/예술영화,영화구분_일반영화
0,14.306190,12.886638,13.607103,1587,17613682,False,False,False,False,False,...,False,False,False,False,False,True,False,False,False,True
1,15.172542,16.604523,13.607103,1978,16264944,False,False,False,False,False,...,False,False,True,False,False,True,False,False,False,True
2,15.714698,16.402393,12.960970,1912,14410754,False,False,False,False,False,...,False,False,False,True,True,False,False,False,False,True
3,15.908342,15.263750,13.607103,966,14245998,False,False,False,False,False,...,False,False,False,False,True,False,False,False,False,True
4,14.740589,15.240150,13.427532,2835,13934592,False,False,False,True,False,...,False,False,False,False,True,False,False,False,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2324,7.217443,7.217443,9.570018,82,1362,True,False,False,False,False,...,False,False,False,False,False,True,False,False,False,True
2325,7.205635,7.205635,7.205635,60,1346,False,False,False,False,False,...,False,False,False,False,False,True,False,False,True,False
2326,7.156956,8.649415,9.092294,71,1282,False,False,False,True,False,...,False,False,False,False,False,True,False,False,False,True
2327,7.770353,7.134891,7.134891,79,1254,False,False,False,False,False,...,False,False,False,False,False,True,False,False,True,False


In [4]:
# 변수 제거 함수
def recursive_feature_elimination(X, y, model, min_features=1, verbose=True):
    # 초기 변수 개수
    n_features = X.shape[1]
    
    # 현재 변수들의 성능
    best_score = float("inf")
    best_features = X.columns.tolist()
    
    # 최소 변수 개수 이상인 경우
    while n_features > min_features:
        # 모든 변수에 대해 반복
        scores = []
        for feature in X.columns:
            # 선택한 변수 제외
            features = X.columns.drop(feature)
            X_new = X[features]
            
            # 모델 학습 및 평가
            model.fit(X_new, y)
            y_pred = model.predict(X_new)
            score = mean_squared_error(y, y_pred)
            scores.append(score)
            
        # 가장 성능이 좋은 변수 선택
        idx = pd.Index(scores).argmin()
        worst_feature = X.columns[idx]
        
        # 변수 제거
        X.drop(worst_feature, axis=1, inplace=True)
        n_features = X.shape[1]
        if verbose:
            print(f"Removing {worst_feature}: {n_features} features left")
        
        # 현재 변수 개수에서의 성능이 최선인 경우
        if min(scores) < best_score:
            best_score = min(scores)
            best_features = X.columns.tolist()
        else:
            break
    
    # 최종 선택된 변수들
    return best_features

# ElasticNetCV 모델 객체 생성
alphas = [0.001, 0.01, 0.1, 1, 10, 100]
enet = ElasticNetCV(alphas=alphas, l1_ratio=0.5, cv=5, max_iter=10000)

# 최소 변수 개수 지정
min_features = 5

# 재귀적 변수 제거 수행
X = m.drop("전국 관객수", axis=1)
y = m["전국 관객수"]
selected_features = recursive_feature_elimination(X, y, enet, min_features=min_features)

# 선택된 변수 출력
print(f"{len(selected_features)} features selected:")
print(selected_features)

Removing 국적_기타: 40 features left
Removing 국적_프랑스: 39 features left
Removing 국적_홍콩: 38 features left
39 features selected:
['감독', '제작사', '배급사', '전국 스크린수', '국적_독일', '국적_러시아', '국적_미국', '국적_스페인', '국적_영국', '국적_일본', '국적_중국', '국적_한국', '국적_홍콩', '장르_SF', '장르_가족', '장르_공연', '장르_공포(호러)', '장르_기타', '장르_다큐멘터리', '장르_드라마', '장르_멜로/로맨스', '장르_뮤지컬', '장르_미스터리', '장르_범죄', '장르_사극', '장르_서부극(웨스턴)', '장르_스릴러', '장르_애니메이션', '장르_액션', '장르_어드벤처', '장르_전쟁', '장르_코미디', '장르_판타지', '등급_12세이상관람가', '등급_15세이상관람가', '등급_전체관람가', '등급_청소년관람불가', '영화구분_독립/예술영화', '영화구분_일반영화']


In [5]:
# 데이터 분할
X = m[selected_features]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [6]:
# 모델 정의
knn = KNeighborsRegressor()
rf = RandomForestRegressor(random_state=42)
gb = GradientBoostingRegressor(random_state=42)
xgb = XGBRegressor(random_state=42)
lgbm = LGBMRegressor(random_state=42)

# 앙상블 모델 정의
stack = StackingCVRegressor(regressors=(knn, rf, gb, xgb, lgbm),
                            meta_regressor=gb,
                            use_features_in_secondary=True)

# 앙상블 모델 학습
stack.fit(X_train, y_train)

# 앙상블 모델 예측
y_pred = stack.predict(X_test)

c:\Users\user\anaconda3\envs\DACON-AI\Lib\site-packages\sklearn\base.py:432: UserWarning: X has feature names, but KNeighborsRegressor was fitted without feature names
  warnings.warn(
c:\Users\user\anaconda3\envs\DACON-AI\Lib\site-packages\sklearn\base.py:432: UserWarning: X has feature names, but RandomForestRegressor was fitted without feature names
  warnings.warn(
c:\Users\user\anaconda3\envs\DACON-AI\Lib\site-packages\sklearn\base.py:432: UserWarning: X has feature names, but GradientBoostingRegressor was fitted without feature names
  warnings.warn(


In [7]:
# 평가지표 계산
rmse = mean_squared_error(y_test, y_pred, squared=False)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

# 평가지표 출력
print(f"RMSE: {rmse}")
print(f"MAE: {mae}")
print(f"R-squared: {r2}")

RMSE: 887754.3613640231
MAE: 377123.1201197584
R-squared: 0.7200149514742675


In [8]:
# KNN 모델의 하이퍼파라미터 탐색 공간과 목적 함수 정의
def knn_objective(trial):
    n_neighbors = trial.suggest_int("n_neighbors", 1, 30)
    knn = KNeighborsRegressor(n_neighbors=n_neighbors)
    score = cross_val_score(knn, X, y, cv=5, scoring="neg_mean_squared_error")
    rmse = (-1 * score.mean()) ** 0.5
    return rmse

# Random Forest 모델의 하이퍼파라미터 탐색 공간과 목적 함수 정의
def rf_objective(trial):
    n_estimators = trial.suggest_int("n_estimators", 100, 1000, step=100)
    max_depth = trial.suggest_int("max_depth", 3, 10)
    rf = RandomForestRegressor(n_estimators=n_estimators, max_depth=max_depth, random_state=42)
    score = cross_val_score(rf, X, y, cv=5, scoring="neg_mean_squared_error")
    rmse = (-1 * score.mean()) ** 0.5
    return rmse

# Gradient Boosting 모델의 하이퍼파라미터 탐색 공간과 목적 함수 정의
def gb_objective(trial):
    n_estimators = trial.suggest_int("n_estimators", 100, 1000, step=100)
    learning_rate = trial.suggest_float("learning_rate", 0.001, 0.1)
    max_depth = trial.suggest_int("max_depth", 3, 10)
    gb = GradientBoostingRegressor(n_estimators=n_estimators, learning_rate=learning_rate,
                                   max_depth=max_depth, random_state=42)
    score = cross_val_score(gb, X, y, cv=5, scoring="neg_mean_squared_error")
    rmse = (-1 * score.mean()) ** 0.5
    return rmse

# XGBoost 모델의 하이퍼파라미터 탐색 공간과 목적 함수 정의
def xgb_objective(trial):
    n_estimators = trial.suggest_int("n_estimators", 100, 1000, step=100)
    learning_rate = trial.suggest_float("learning_rate", 0.001, 0.1)
    max_depth = trial.suggest_int("max_depth", 3, 10)
    subsample = trial.suggest_float("subsample", 0.5, 1)
    xgb = XGBRegressor(n_estimators=n_estimators, learning_rate=learning_rate,
                       max_depth=max_depth, subsample=subsample, random_state=42)
    score = cross_val_score(xgb, X, y, cv=5, scoring="neg_mean_squared_error")
    rmse = (-1 * score.mean()) ** 0.5
    return rmse

# LightGBM 모델의 하이퍼파라미터 탐색 공간과 목적 함수 정의
def lgbm_objective(trial):
    n_estimators = trial.suggest_int("n_estimators", 100, 1000, step=100)
    learning_rate = trial.suggest_float("learning_rate", 0.001, 0.1)
    max_depth = trial.suggest_int("max_depth", 3, 10)
    num_leaves = trial.suggest_int("num_leaves", 10, 100)
    lgbm = LGBMRegressor(n_estimators=n_estimators, learning_rate=learning_rate,
    max_depth=max_depth, num_leaves=num_leaves, random_state=42)
    score = cross_val_score(lgbm, X, y, cv=5, scoring="neg_mean_squared_error")
    rmse = (-1 * score.mean()) ** 0.5
    return rmse

def optimize_model(model_objective, n_trials=100):
    study = optuna.create_study(direction="minimize", sampler=optuna.samplers.CmaEsSampler())
    n_jobs = multiprocessing.cpu_count()  # 사용 가능한 모든 CPU 코어 수
    study.optimize(model_objective, n_trials=n_trials, n_jobs=n_jobs)
    print(f"Best RMSE: {study.best_value:.4f}")
    print(f"Best Parameters: {study.best_params}")
    return study.best_params

# KNN 모델의 하이퍼파라미터 최적화
knn_params = optimize_model(knn_objective)

# Random Forest 모델의 하이퍼파라미터 최적화
rf_params = optimize_model(rf_objective)

# Gradient Boosting 모델의 하이퍼파라미터 최적화
gb_params = optimize_model(gb_objective)

# XGBoost 모델의 하이퍼파라미터 최적화
xgb_params = optimize_model(xgb_objective)

# LightGBM 모델의 하이퍼파라미터 최적화
lgbm_params = optimize_model(lgbm_objective)

[I 2023-05-08 15:35:25,130] A new study created in memory with name: no-name-cc31b7e6-666b-4da2-baa3-70b4deb7a5d9
[I 2023-05-08 15:35:25,546] Trial 0 finished with value: 1993644.6043167887 and parameters: {'n_neighbors': 24}. Best is trial 0 with value: 1993644.6043167887.
[I 2023-05-08 15:35:25,550] Trial 1 finished with value: 1980672.0450931801 and parameters: {'n_neighbors': 3}. Best is trial 1 with value: 1980672.0450931801.
[I 2023-05-08 15:35:25,567] `CmaEsSampler` only supports two or more dimensional continuous search space. `RandomSampler` is used instead of `CmaEsSampler`.
[I 2023-05-08 15:35:25,581] `CmaEsSampler` only supports two or more dimensional continuous search space. `RandomSampler` is used instead of `CmaEsSampler`.
[I 2023-05-08 15:35:25,584] Trial 5 finished with value: 1985302.6842324692 and parameters: {'n_neighbors': 2}. Best is trial 1 with value: 1980672.0450931801.
[I 2023-05-08 15:35:25,632] `CmaEsSampler` only supports two or more dimensional continuous

Best RMSE: 1980672.0451
Best Parameters: {'n_neighbors': 3}


[I 2023-05-08 15:35:45,180] Trial 3 finished with value: 1849272.5667385417 and parameters: {'n_estimators': 200, 'max_depth': 3}. Best is trial 3 with value: 1849272.5667385417.
[I 2023-05-08 15:35:46,245] Trial 0 finished with value: 1843914.587709778 and parameters: {'n_estimators': 100, 'max_depth': 8}. Best is trial 0 with value: 1843914.587709778.
[I 2023-05-08 15:35:51,053] Trial 7 finished with value: 1849679.6302838966 and parameters: {'n_estimators': 300, 'max_depth': 3}. Best is trial 0 with value: 1843914.587709778.
[I 2023-05-08 15:35:51,377] Trial 4 finished with value: 1849679.6302838966 and parameters: {'n_estimators': 300, 'max_depth': 3}. Best is trial 0 with value: 1843914.587709778.
[I 2023-05-08 15:36:07,570] Trial 2 finished with value: 1839405.8052794454 and parameters: {'n_estimators': 300, 'max_depth': 7}. Best is trial 2 with value: 1839405.8052794454.
[I 2023-05-08 15:36:15,389] Trial 1 finished with value: 1827007.6891279952 and parameters: {'n_estimators': 

Best RMSE: 1824480.6960
Best Parameters: {'n_estimators': 500, 'max_depth': 5}


[I 2023-05-08 15:45:50,747] Trial 2 finished with value: 1951886.006285264 and parameters: {'n_estimators': 100, 'learning_rate': 0.006436811581816698, 'max_depth': 7}. Best is trial 2 with value: 1951886.006285264.
[I 2023-05-08 15:45:54,284] Trial 7 finished with value: 1811623.4014803285 and parameters: {'n_estimators': 300, 'learning_rate': 0.014728315174794646, 'max_depth': 3}. Best is trial 7 with value: 1811623.4014803285.
[I 2023-05-08 15:46:17,184] Trial 3 finished with value: 1837370.5049647035 and parameters: {'n_estimators': 500, 'learning_rate': 0.02911004913553306, 'max_depth': 5}. Best is trial 7 with value: 1811623.4014803285.
[I 2023-05-08 15:46:24,508] Trial 4 finished with value: 1860528.8024387585 and parameters: {'n_estimators': 300, 'learning_rate': 0.025822597827790356, 'max_depth': 10}. Best is trial 7 with value: 1811623.4014803285.
[I 2023-05-08 15:46:41,085] Trial 1 finished with value: 1897737.1944872702 and parameters: {'n_estimators': 400, 'learning_rate':

Best RMSE: 1808599.7644
Best Parameters: {'n_estimators': 400, 'learning_rate': 0.044113342131720704, 'max_depth': 3}


[I 2023-05-08 15:53:22,432] Trial 2 finished with value: 1829636.1143199685 and parameters: {'n_estimators': 100, 'learning_rate': 0.08290902106187448, 'max_depth': 4, 'subsample': 0.7203520723924421}. Best is trial 2 with value: 1829636.1143199685.
[I 2023-05-08 15:53:27,477] Trial 4 finished with value: 1837408.1046434373 and parameters: {'n_estimators': 200, 'learning_rate': 0.09070665818191803, 'max_depth': 4, 'subsample': 0.7922528735737843}. Best is trial 2 with value: 1829636.1143199685.
[I 2023-05-08 15:53:29,142] Trial 6 finished with value: 1818282.731650317 and parameters: {'n_estimators': 300, 'learning_rate': 0.07991173260039851, 'max_depth': 3, 'subsample': 0.9600517233185744}. Best is trial 6 with value: 1818282.731650317.
[I 2023-05-08 15:53:29,342] Trial 5 finished with value: 1808600.3986311068 and parameters: {'n_estimators': 300, 'learning_rate': 0.023951784670726534, 'max_depth': 3, 'subsample': 0.5516847940665125}. Best is trial 5 with value: 1808600.3986311068.
[

Best RMSE: 1806642.6067
Best Parameters: {'n_estimators': 200, 'learning_rate': 0.032589494199244534, 'max_depth': 3, 'subsample': 0.7077421050042759}


[I 2023-05-08 15:59:34,260] Trial 2 finished with value: 1819520.3741036388 and parameters: {'n_estimators': 100, 'learning_rate': 0.08797678613262665, 'max_depth': 8, 'num_leaves': 64}. Best is trial 2 with value: 1819520.3741036388.
[I 2023-05-08 15:59:34,612] Trial 4 finished with value: 1824203.4788601662 and parameters: {'n_estimators': 200, 'learning_rate': 0.08492983791553775, 'max_depth': 6, 'num_leaves': 82}. Best is trial 2 with value: 1819520.3741036388.
[I 2023-05-08 15:59:35,648] Trial 6 finished with value: 1822345.6036371551 and parameters: {'n_estimators': 200, 'learning_rate': 0.08823357005907552, 'max_depth': 8, 'num_leaves': 65}. Best is trial 2 with value: 1819520.3741036388.
[I 2023-05-08 15:59:36,708] Trial 0 finished with value: 1829968.0689338858 and parameters: {'n_estimators': 300, 'learning_rate': 0.0727544003755557, 'max_depth': 10, 'num_leaves': 17}. Best is trial 2 with value: 1819520.3741036388.
[I 2023-05-08 15:59:36,981] Trial 3 finished with value: 182

Best RMSE: 1807287.2212
Best Parameters: {'n_estimators': 400, 'learning_rate': 0.009849483264241562, 'max_depth': 6, 'num_leaves': 32}


In [9]:
# 앙상블 모델 정의
stack = StackingCVRegressor(regressors=(knn, rf, gb, xgb, lgbm),
                            meta_regressor=gb,
                            use_features_in_secondary=True)

# 앙상블 모델 학습
stack.fit(X_train, y_train)

# 앙상블 모델 예측
y_pred = stack.predict(X_test)

c:\Users\user\anaconda3\envs\DACON-AI\Lib\site-packages\sklearn\base.py:432: UserWarning: X has feature names, but KNeighborsRegressor was fitted without feature names
  warnings.warn(
c:\Users\user\anaconda3\envs\DACON-AI\Lib\site-packages\sklearn\base.py:432: UserWarning: X has feature names, but RandomForestRegressor was fitted without feature names
  warnings.warn(
c:\Users\user\anaconda3\envs\DACON-AI\Lib\site-packages\sklearn\base.py:432: UserWarning: X has feature names, but GradientBoostingRegressor was fitted without feature names
  warnings.warn(


In [10]:
# 평가지표 계산
rmse = mean_squared_error(y_test, y_pred, squared=False)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

# 평가지표 출력
print(f"RMSE: {rmse}")
print(f"MAE: {mae}")
print(f"R-squared: {r2}")

RMSE: 918809.8994365718
MAE: 387231.98269588564
R-squared: 0.700083373466783
